In [617]:
import numpy as np
import matplotlib.pyplot as plt

In [618]:
def linear_interpolation(x0, y0, x1, y1, x):
    """
    Perform linear interpolation to find the y value corresponding to x.

    Parameters:
    x0, y0: coordinates of the first point
    x1, y1: coordinates of the second point
    x: the x value to interpolate

    Returns:
    y: the interpolated y value
    """
    y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
    return y

# Example usage:
# x0, y0 = 0, 0
# x1, y1 = 10, 10
# x = 5
# y = linear_interpolation(x0, y0, x1, y1, x)
# print(f"The interpolated value at x={x} is y={y}")

## Given Constants

In [619]:
gear = "helix"


n_p = 1200

n_g_guess = 387

N_p = 19 # between 17 and 20 - only for spur?, idk abt helical
N_g = None

P_d = 12
P_t = None
P_nd = None

D_p = None
D_g = None


DEFINED_FACE_WIDTH = False
F = 1.5

P = 5 # Power in HP
life_time = 15000 # in hours?


A_v = 10



In [620]:
#helical
phi_n = None
phi_t = 20

psi = 25

In [621]:
#helical
if phi_n is None:
    phi_n = 180 / np.pi * np.arctan(np.tan(phi_t * np.pi / 180) * np.cos(psi * np.pi / 180))
elif phi_t is None:
    phi_t = 180 / np.pi * np.arctan(np.tan(phi_n * np.pi / 180) / np.cos(psi * np.pi / 180))

phi_n, phi_t

(18.256124010900294, 20)

NEED TO CALCULATE GEAR PROPERLY FIRST

Calculate P_d, diameters etc

In [622]:
if P_d is None and P_nd is not None:
    P_d = P_nd * np.cos(psi * np.pi / 180)
    D_p = N_p / P_d
    D_g = N_g / P_d
    assert(N_g is not None)


elif P_d is None and D_p is not None:
    P_d = N_p / D_p
    

elif P_nd is None and D_p is None:
    P_nd = P_d / np.cos(psi * np.pi / 180)
    D_p = N_p / P_d
    D_g = N_g / P_d
    assert(N_g is not None)
    
assert(P_d is not None)

P_t = np.pi / P_d

P_x = np.pi / (P_d * np.tan(psi * np.pi / 180) )

assert(P_x - P_t / np.tan(psi * np.pi / 180) < 10**-8)

P_d, P_x, P_t


(12, 0.5614305989204417, 0.2617993877991494)

Face width

In [623]:
#face width

if not DEFINED_FACE_WIDTH:
    F = 2 * P_x #nominal value

F_div_P_x = F / P_x

# assert(F_div_P_x - 2 > 10**-8)

F, F_div_P_x


(1.1228611978408833, 2.0)

## Calculations

In [624]:
#shock factor
k_o = 1.5

In [625]:
P_des = P * k_o
P_des

7.5

### Velocity ratio, output speed

In [626]:
##Velocity ratio

VR_guess = n_p/n_g_guess


In [627]:
##Number of teeth on gear

N_g = round(N_p * VR_guess)
N_g


59

In [628]:
VR = N_g/N_p
n_g = n_p / VR

VR, n_g

(3.1052631578947367, 386.4406779661017)

### Diameters, Center distances, pitch line speed

In [629]:
##Center distance

C = (N_p + N_g) / (2 * P_d)
C

3.25

In [630]:
##v_t

v_t = np.pi * D_p * n_p / 12
v_t

497.41883681838385

## Helical gear stuff follows

### Calculate J, I

In [631]:
#find J_p and J_G

J_p = 0.405
J_G = 1000000000000000000

I = 0.19

assert(I > 0); assert(J_p > 0); assert(J_G > 0)

### Calculate forces

In [632]:
## W_t - helical gear

W_t = 33000 * P / v_t
W_r = W_t * np.tan(phi_t * np.pi / 180)
Wx = W_t * np.tan(psi * np.pi / 180)
W_t, W_r, Wx

(331.71240770731873, 120.73344274223882, 154.6800360189559)

## choose a material - this part is sketch

seems to apply the same for all gears??

this part consists of finding numbers in tables

In [633]:
C_p = 2300 # for steel

In [634]:
# usually given - use table 9-5 in mott 
# A_v = 10
assert(A_v is not None)


In [635]:
k_v = 1.25 #from A_v

assert(k_v > 0)

In [636]:
#C_pf
C_pf = None

if F / D_p < 0.5:
    C_pf = -1
    # assert(False)
    #use graph
elif F <= 1.0:
    C_pf = F / (10 * D_p) - 0.025
elif F <= 15:
    C_pf = F / (10 * D_p) - 0.0375 + 0.0125 * F

assert(C_pf is not None)

C_pf


0.047453314310329994

In [637]:
#C_ma
cond = "commercial Enclosed".upper()
assert(cond is not None)

C_ma = -1

if cond == "commercial enclosed".upper():
    C_ma = 0.127 + 0.0158 * F - 1.093 * 10**-4 * F**2
elif cond == "precision".upper():
    C_ma = 0.0675 + 0.0128 * F - 0.926 * 10**-4 * F**2
elif cond == "extra".upper():
    C_ma = 0.0380 + 0.0102 * F - 0.822 * 10**-4 * F**2

C_ma


0.14460339959831686

In [638]:
k_m = 1 + C_ma + C_pf
k_m

1.1920567139086469

### Find $k_s, k_B, k_R, k_v, SF$ from a table idrk

In [639]:
k_s = 1.0

In [640]:
#k_B
# assert(False)
#idk how to determine this
k_B = 1.00
k_B

1.0

In [641]:
k_R = 1

In [642]:
safety_factor = 1.0

## Calculate $Y_{np}, Y_{ng}$ with the above numbers

### Number of loading cycles

In [643]:
#Number of cycles

N_cp = 60 * life_time * n_p
N_cg = 60 * life_time * n_g

print(f"N_cp: {N_cp:.2e}, N_cg: {N_cg:.2e}")

N_cp: 1.08e+09, N_cg: 3.48e+08


### Calc $Y_{xx}, Z_{xx}$

In [644]:
# assert(False)
Y_np = 0.95
Y_ng = 1.0

Z_np = 0.898
Z_ng = 0.922

## Calculate $S_{tP}$ and $S_{tG}$

In [645]:
assert(F != 0)
assert(J_p != 0)

s_tp = (W_t * P_d) / (F * J_p) * k_o * k_s * k_m * k_B * k_v
s_tp

# W_t, P_d, F, J_p, k_o, k_s, k_m, k_B, k_v
print(f"{W_t=}, {P_d=}, {F=}, {J_p=}, {k_o=}, {k_s=}, {k_m=}, {k_B=}, {k_v=}")
s_tp

W_t=331.71240770731873, P_d=12, F=1.1228611978408833, J_p=0.405, k_o=1.5, k_s=1.0, k_m=1.1920567139086469, k_B=1.0, k_v=1.25


19564.108163772114

In [646]:
s_tG = s_tp * J_p / J_G
s_tG

7.923463806327708e-15

In [647]:
#idk what this is
s_atP_Greater_than = s_tp * k_R * safety_factor / Y_np
s_atP_Greater_than

20593.798067128544

In [1]:
#idk what this is
s_atG_Greater_than = s_tG * k_R * safety_factor / Y_ng
s_atG_Greater_than



NameError: name 's_tG' is not defined

### Calculate $S_c$

In [649]:
S_c = C_p * np.sqrt( (W_t * k_o * k_s * k_m  * k_v) / (F * D_p * I) ) #
S_c

107753.59643395903

### Find adjusted values of $S_c$

In [650]:
S_acP_Greater_than = S_c * (k_R * safety_factor) / Z_np
S_acP_Greater_than

119992.86908013254

In [651]:
S_acg_Greater_than = S_c * (k_R * safety_factor) / Z_ng
S_acg_Greater_than

116869.41044897941

### Calculate the required brinell hardness

In [652]:
# Prompt the user for input
HB_grade = input("HB grade 1 or 2? ")

Contact_HB_p = 0
Bending_HB_p = 0

if int(HB_grade) == 1:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 12.8 ) / 0.0773

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 29.1 ) / 0.322
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 12.8 ) / 0.0773

elif int(HB_grade) == 2:
    Contact_HB_p = ( S_acP_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_p = ( s_atP_Greater_than / 1000 - 16.4 ) / 0.102

    Contact_HB_g = ( S_acg_Greater_than / 1000 - 34.3 ) / 0.349
    Bending_HB_g = ( s_atG_Greater_than / 1000 - 16.4 ) / 0.102

else:
    assert(False)


print(f"Contact_HB_p: {Contact_HB_p:.2f} psi \nBending_HB_p: {Bending_HB_p:.2f} psi\n")

print(f"Contact_HB_g: {Contact_HB_g:.2f} psi \nBending_HB_g: {Bending_HB_g:.2f} psi")

Contact_HB_p: 282.28 psi 
Bending_HB_p: 100.83 psi

Contact_HB_g: 272.58 psi 
Bending_HB_g: -165.59 psi


Use tables in the bible to find an appropriate material that satisfies the required HB. The material should be the same for the gear and the pinion.

Make sure to use table 9-8 form Mott to determine the heat treatment

Get $S_{acp}, S_{atp}$ for the material

In [653]:
S_acp = 126000#None
S_atp = 36100#None

S_acg = 1#None
S_atg = 1#None

In [654]:
contact_SF_p = S_acp * Y_np / (S_c * k_R)
bending_SF_p = S_atp * Y_np / (s_tp * k_R)

print(f"Contact Safety Factor (Pinion): {contact_SF_p:f} \nBending Safety Factor (Pinion): {bending_SF_p:f}\n")

contact_SF_g = S_acg * Y_np / (S_c * k_R)
bending_SF_g = S_atg * Y_np / (s_tG * k_R)

print(f"Contact Safety Factor (Gear): {contact_SF_g:.2f} \nBending Safety Factor (Gear): {bending_SF_g:.2f}")

print(S_acp, Y_np, S_c, k_R)


Contact Safety Factor (Pinion): 1.110868 
Bending Safety Factor (Pinion): 1.752955

Contact Safety Factor (Gear): 0.00 
Bending Safety Factor (Gear): 119897058057023.80
126000 0.95 107753.59643395903 1


In [655]:
# Power transmitting capacity

P_cap = n_p * F * I / (126000 * k_o * k_s * k_m * k_v) * ( (S_acP_Greater_than * D_p * Z_np) / (safety_factor * k_R * C_p) ) ** 2

print(f"P_cap = {P_cap:.4f} HP")

P_cap = 5.0020 HP


# Final Design

In [656]:
variables = {

    "D_p": f"{D_p} in",
    "--- ": "---",
    "F": f"{F} in",
    "--- ": "---",
    "N_p": f"{N_p} teeth",
    "P_d": f"{P_d} teeth/in",
    "n_p": f"{n_p} rpm",
    "--- " : "---",
    "D_g" : f"{D_g} in",
    "N_g": f"{N_g} teeth",
    "P": f"{P} HP",
    "P_nd": f"{P_nd:.2f} teeth/in",
    "P_x": f"{P_x:.2f} in",
    "P_t": f"{P_t:.2f} in",
    "VR": f"{VR:.2f}",
    "W_r": f"{W_r:.2f} lbs",
    "W_t": f"{W_t:.2f} lbs",
    "Wx": f"{Wx:.2f} lbs",
    "k_o": f"{k_o}",
    "k_v": f"{k_v}",
    "life_time": f"{life_time} hours",
    "n_g": f"{n_g:.2f} rpm",
    "phi_n": f"{phi_n} degrees",
    "phi_t": f"{phi_t:.2f} degrees",
    "psi": f"{psi} degrees",
    "v_t": f"{v_t:.2f} ft/min"
}

for var_name, var_value in variables.items():
    print(f"{var_name}: {var_value}")

D_p: 1.5833333333333333 in
--- : ---
F: 1.1228611978408833 in
N_p: 19 teeth
P_d: 12 teeth/in
n_p: 1200 rpm
D_g: 0.0 in
N_g: 59 teeth
P: 5 HP
P_nd: 13.24 teeth/in
P_x: 0.56 in
P_t: 0.26 in
VR: 3.11
W_r: 120.73 lbs
W_t: 331.71 lbs
Wx: 154.68 lbs
k_o: 1.5
k_v: 1.25
life_time: 15000 hours
n_g: 386.44 rpm
phi_n: 18.256124010900294 degrees
phi_t: 20.00 degrees
psi: 25 degrees
v_t: 497.42 ft/min


In [657]:
C

3.25